In [1]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, DensityMatrix, Pauli, Operator, random_statevector
from qiskit_aer import AerSimulator, Aer
import numpy as np
from scipy.linalg import sqrtm
import cvxpy as cp
from IPython.display import clear_output
from winsound import Beep

def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

def check_legal(matrix, print_errors=True):
    errors, legal = [], True
    if not hermitian(matrix):
        errors.append('not hermitian')
    if not trace_one(matrix):
        errors.append('trace not equal to one')
    if not positive_semi_definite(matrix):
        errors.append('not positive semidefinite')
    if len(errors) > 0:
        legal = False
    if print_errors:
        if not legal:
            print(f'input is not legal: ' + '; '.join(errors))
        else: 
            print('input is a legal density matrix')
    return legal
        
        
def generate_prob_lst(num_states):
    prob_lst = np.array([np.random.random() for _ in range(num_states)])
    prob_lst /= np.sum(prob_lst)
    return prob_lst

def get_rank(dm, tol=1e-10):
    return int(np.sum(np.linalg.eigvalsh(dm) > tol))

def get_fidelity(dm1, dm2, tol=1e-5):
    # assert is_legal(dm1) and is_legal(dm2), 'inputs are not legal density matrices'
    if not is_legal(dm1) and is_legal(dm2):
        print("Warning: inputs are not legal density matrices")
    try: 
        fidelity = (np.trace(sqrtm(sqrtm(dm1) @ dm2 @ sqrtm(dm1)))) ** 2
    except ValueError:
        print('fidelity cannot be computed for given inputs')
    assert np.abs(np.imag(fidelity)) < tol, 'fidelity is not real within tol'
    return fidelity.real

def generate_dm(num_qubits, num_states, state_lst=None, prob_lst=None):
    if state_lst is None:
        state_lst = [random_statevector(2**num_qubits) for _ in range(num_states)]
    if prob_lst is None:
        prob_lst = generate_prob_lst(num_states)
    density_matrix = sum([DensityMatrix(state_lst[i]).data * prob_lst[i] for i in range(num_states)])
    return density_matrix

def generate_Pauli_strings(num_strings, length, contain_I=True):
    if contain_I:
        characters = ['I', 'X', 'Y', 'Z']
    else:
        characters = ['X', 'Y', 'Z']
    generated_strings = []
    assert num_strings < len(characters) ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != 'I' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_Pauli_expectations(dm, obsv):
    return np.trace(dm @ Pauli(obsv).to_matrix()).real

def get_trace_norm(dm):
    return np.sum(np.linalg.svd(dm, compute_uv=False))

In [4]:
# Prepare a state for measurement
state = generate_dm(6, 2, prob_lst=np.array([.99, .01]))
r = get_rank(state)
d = 2 ** 6

# Prepare observables
c = 1
num_measurements = int(c * r * d * (np.log(d)) ** 2)
observables = generate_Pauli_strings(num_measurements, 6, contain_I=True)
expectations = [generate_Pauli_expectations(state, obsv) for obsv in observables]

In [5]:
def optimize(dim, obsv, expct, tol=1e-5):
    sigma = cp.Variable((dim, dim), complex=True)
    objective = cp.Minimize(cp.abs(cp.norm(sigma, 'nuc')))
    constraints = [cp.real(cp.trace(sigma)) == 1]
    for o, e in zip(obsv, expct):
        constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    return sigma.value
    
    
sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')
Beep(1000, 2000)

optimal
fidelity between outcome and target is 0.9997987499940038


In the case of
`state = generate_dm(6, 2, prob_lst=np.array([.99, .01]))`:

c = 1
tol = 1e-5

In [2]:
# Prepare a state for measurement
state = generate_dm(7, 5, prob_lst=np.array([.96, .01, .01, .01, .01]))
r = np.sum(np.linalg.eigvals(state) > 2e-2)
# print(r)
d = 2 ** 7

# Prepare observables
c = .7
num_measurements = int(c * r * d * (np.log(d)) ** 2)
observables = generate_Pauli_strings(num_measurements, 7, contain_I=True)
expectations = [generate_Pauli_expectations(state, obsv) for obsv in observables]
# Beep(1000, 2000)

In [7]:
def optimize(dim, obsv, expct, tol=1e-5):
    sigma = cp.Variable((dim, dim), complex=True)
    objective = cp.Minimize(cp.abs(cp.norm(sigma, 'nuc')))
    constraints = [cp.real(cp.trace(sigma)) == 1]
    for o, e in zip(obsv, expct):
        constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    return sigma.value
    
    
sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')
Beep(1000, 2000)

optimal
fidelity between outcome and target is 0.9996405894552405


it seems `c=1` doesn't depend on number of qubits

In [ ]:
def optimize(dim, obsv, expct, tol=1e-5):
    sigma = cp.Variable((dim, dim), complex=True)
    objective = cp.Minimize(cp.abs(cp.norm(sigma, 'nuc')))
    constraints = [cp.real(cp.trace(sigma)) == 1]
    for o, e in zip(obsv, expct):
        constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    return sigma.value
    
    
sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')
Beep(1000, 2000)

In [3]:
!pip install ecos


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install mosek

   ---------------------------------------- 0.0/20.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/20.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/20.0 MB ? eta -:--:--
   - -------------------------------------- 0.8/20.0 MB 3.4 MB/s eta 0:00:06
   ---- ----------------------------------- 2.1/20.0 MB 5.9 MB/s eta 0:00:04
   ------------- -------------------------- 6.6/20.0 MB 10.3 MB/s eta 0:00:02
   ------------------------ --------------- 12.1/20.0 MB 14.2 MB/s eta 0:00:01
   ------------------------------------ --- 18.1/20.0 MB 17.0 MB/s eta 0:00:01
   ---------------------------------------- 20.0/20.0 MB 16.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install cvxopt

   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.8 MB 3.3 MB/s eta 0:00:04
   --------- ------------------------------ 3.1/12.8 MB 7.4 MB/s eta 0:00:02
   --------------------------- ------------ 8.9/12.8 MB 14.6 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 16.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cvxpy as cp
import numpy as np

def optimize(dim, obsv, expct, tol=1e-5):
    # Define the variable
    sigma = cp.Variable((dim, dim), complex=True)
    
    # Define the objective function: minimize the nuclear norm
    objective = cp.Minimize(cp.norm(sigma, 'nuc'))
    
    # Define the constraints
    constraints = [cp.real(cp.trace(sigma)) == 1]
    for o, e in zip(obsv, expct):
        # Add constraints for each observation and expected value
        constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
    
    # Define and solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.CVXOPT)  # You can choose other solvers like cp.SCS, cp.MOSEK, cp.CVXOPT, etc.
    
    # Check the status and return the result
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        print("Optimization successful:", problem.status)
        return sigma.value
    else:
        raise ValueError('Optimization failed:' + str(problem.status))

sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')
Beep(1000, 2000)


: 